# 0.1. imports

In [2]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
import numpy as np
# deep copy
import copy

# 0.2. config

In [3]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.1.3. load `pylcaio`

In [4]:
sys.path.append(str(Path(Path.home(), config['path_dir_repo_pylcaio'])))
import pylcaio

# 0.2. file paths
## 0.2.1. directories

In [5]:
%%capture
print(path_dir_data := Path(Path.home(), config['path_dir_data']))
print(path_dir_data_processed := Path(path_dir_data, config['path_dir_data_processed']))

## 0.2.2. files

In [6]:
%%capture
print(path_file_e2m_pickle := Path(path_dir_data_processed, (config['ecoinvent_version_name'] + config['e2m_pickle_filename'])))

# 1. load pickled `Ecoinvent` data

In [7]:
with gzip.open(path_file_e2m_pickle, 'rb') as file_in:
    ecoinvent: dict = pd.read_pickle(file_in)

In [8]:
[*ecoinvent.keys()]

['PRO',
 'STR',
 'IMP',
 'A',
 'F',
 'C',
 'PRO_header',
 'STR_header',
 'IMP_header',
 'processingdata']

# 2. investigation

## 2.1. `PRO` "process labels table"

In [9]:
ecoinvent['PRO'].shape

(16022, 23)

In [10]:
ecoinvent['PRO'].head(5)

,activityId,productId,activityName,ISIC,price,priceUnit,EcoSpoldCategory,geography,technologyLevel,macroEconomicScenario,...,productName,unitName,cpc,dry mass [kg]_y,wet mass [kg]_y,activityNameId,activityType,startDate,endDate,activityName_duplicate
index,,,,,,,,,,,,,,,,,,,,,
a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,a96cb241-a4a9-4980-a16a-ba4b6a80175e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CA-QC,Current,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production"
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production"
d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,d1c3b8ad-58b6-4ea4-8877-a51b356556bc,281fc4f0-c05d-410a-a784-06e3508e78e6,"barley grain, feed production, Swiss integrate...","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CH,Current,Business-as-Usual,...,"barley grain, feed, Swiss integrated production",kg,23319: Preparations used in animal feeding n.e.c.,0.85,1,938a1c4e-41f2-40ea-9bf1-9f09c2437798,0,1996-01-01,2018-12-31,"barley grain, feed production, Swiss integrate..."
92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,92068396-88c7-45ed-9008-622008a299f3,0d860eb4-1a25-41b4-a821-81f5726d86e5,"barley grain, feed production, organic","0111:Growing of cereals (except rice), legumin...",0.159,EUR2005,agricultural means of production/feed,CH,Current,Business-as-Usual,...,"barley grain, feed, organic",kg,23319: Preparations used in animal feeding n.e.c.,0.85,1,6a49a0c6-c114-4471-b6c4-8aa4006e4b0e,0,1996-01-01,2018-12-31,"barley grain, feed production, organic"
35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c,0d860eb4-1a25-41b4-a821-81f5726d86e5,"barley grain, feed production, organic","0111:Growing of cereals (except rice), legumin...",0.159,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,"barley grain, feed, organic",kg,23319: Preparations used in animal feeding n.e.c.,0.85,1,6a49a0c6-c114-4471-b6c4-8aa4006e4b0e,0,1996-01-01,2018-12-31,"barley grain, feed production, organic"


In [11]:
PRO: pd.DataFrame = ecoinvent['PRO']

In [13]:
PRO[PRO['productId'] == 'aeaf5266-3f9c-4074-bd34-eba76a61760c']

,activityId,productId,activityName,ISIC,price,priceUnit,EcoSpoldCategory,geography,technologyLevel,macroEconomicScenario,...,productName,unitName,cpc,dry mass [kg]_y,wet mass [kg]_y,activityNameId,activityType,startDate,endDate,activityName_duplicate
index,,,,,,,,,,,,,,,,,,,,,
a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,a96cb241-a4a9-4980-a16a-ba4b6a80175e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,CA-QC,Current,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production"
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"barley grain, feed production","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,RoW,Current,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,2816d5b0-50fd-42fd-9bc9-aed0bd1913e7,0,1996-01-01,2018-12-31,"barley grain, feed production"
a48bb3d8-1b50-4e46-aa51-81872218007e_aeaf5266-3f9c-4074-bd34-eba76a61760c,a48bb3d8-1b50-4e46-aa51-81872218007e,aeaf5266-3f9c-4074-bd34-eba76a61760c,"market for barley grain, feed","0111:Growing of cereals (except rice), legumin...",0.124,EUR2005,agricultural means of production/feed,GLO,Undefined,Business-as-Usual,...,"barley grain, feed",kg,23319: Preparations used in animal feeding n.e.c.,0.86,1,ae8dac8a-deb7-41f2-aec5-937154b7fccb,1,1996-01-01,2018-12-31,"market for barley grain, feed"


In [16]:
PRO[PRO.duplicated(subset = ['activityId'], keep = 'first')].head(5)

,activityId,productId,activityName,ISIC,price,priceUnit,EcoSpoldCategory,geography,technologyLevel,macroEconomicScenario,...,productName,unitName,cpc,dry mass [kg]_y,wet mass [kg]_y,activityNameId,activityType,startDate,endDate,activityName_duplicate
index,,,,,,,,,,,,,,,,,,,,,
b45ebf7e-9c37-43af-8031-3d52ff5dfe8f_a235b2ff-3237-44b0-a445-b852376a1939,b45ebf7e-9c37-43af-8031-3d52ff5dfe8f,a235b2ff-3237-44b0-a445-b852376a1939,"barley production, Swiss integrated production...","0111:Growing of cereals (except rice), legumin...",0.117,EUR2005,NaN,CH,Current,Business-as-Usual,...,"barley grain, Swiss integrated production",kg,"01152: Barley, other",0.85,1,ce21e796-ac1a-4fcf-a8b4-8170c039516c,0,1996-01-01,2018-12-31,"barley production, Swiss integrated production..."
bac2ed16-1594-40fb-9661-870b3211e4a1_d7f544d6-c372-4fc3-81ae-44aa3614c9fc,bac2ed16-1594-40fb-9661-870b3211e4a1,d7f544d6-c372-4fc3-81ae-44aa3614c9fc,"barley production, Swiss integrated production...","0111:Growing of cereals (except rice), legumin...",0.012,EUR2005,NaN,CH,Current,Business-as-Usual,...,straw,kg,"01913: Cereal straw, husks, unprepared, ground...",0.85,1,698d743b-6647-487d-8319-048b98b32005,0,1996-01-01,2018-12-31,"barley production, Swiss integrated production..."
d44cb0b1-c1ea-4998-b48d-1f09c22b1d8d_f467c4d0-ea1c-4ae3-8d69-712598a0478a,d44cb0b1-c1ea-4998-b48d-1f09c22b1d8d,f467c4d0-ea1c-4ae3-8d69-712598a0478a,"barley production, organic","0111:Growing of cereals (except rice), legumin...",0.152,EUR2005,agricultural production/plant production,RoW,Current,Business-as-Usual,...,"barley grain, organic",kg,"01152: Barley, other",0.85,1,f1c36545-2a83-4e4b-9b4e-78bedae9d81e,0,1996-01-01,2018-12-31,"barley production, organic"
6469db8c-1e2a-4ead-a1c9-f71f2aef2f39_f467c4d0-ea1c-4ae3-8d69-712598a0478a,6469db8c-1e2a-4ead-a1c9-f71f2aef2f39,f467c4d0-ea1c-4ae3-8d69-712598a0478a,"barley production, organic","0111:Growing of cereals (except rice), legumin...",0.152,EUR2005,NaN,CH,Current,Business-as-Usual,...,"barley grain, organic",kg,"01152: Barley, other",0.85,1,f1c36545-2a83-4e4b-9b4e-78bedae9d81e,0,1996-01-01,2018-12-31,"barley production, organic"
57656625-88b2-456f-a7ea-0de734154a40_d7f544d6-c372-4fc3-81ae-44aa3614c9fc,57656625-88b2-456f-a7ea-0de734154a40,d7f544d6-c372-4fc3-81ae-44aa3614c9fc,oat production,"0111:Growing of cereals (except rice), legumin...",0.012,EUR2005,NaN,Canada without Quebec,Current,Business-as-Usual,...,straw,kg,"01913: Cereal straw, husks, unprepared, ground...",0.85,1,033d4d32-d8e9-4455-a6da-644c56bd5ec4,0,2009-01-01,2018-12-31,oat production


In [18]:
PRO[PRO['activityId'] == 'b45ebf7e-9c37-43af-8031-3d52ff5dfe8f']

,activityId,productId,activityName,ISIC,price,priceUnit,EcoSpoldCategory,geography,technologyLevel,macroEconomicScenario,...,productName,unitName,cpc,dry mass [kg]_y,wet mass [kg]_y,activityNameId,activityType,startDate,endDate,activityName_duplicate
index,,,,,,,,,,,,,,,,,,,,,
b45ebf7e-9c37-43af-8031-3d52ff5dfe8f_d7f544d6-c372-4fc3-81ae-44aa3614c9fc,b45ebf7e-9c37-43af-8031-3d52ff5dfe8f,d7f544d6-c372-4fc3-81ae-44aa3614c9fc,"barley production, Swiss integrated production...","0111:Growing of cereals (except rice), legumin...",0.012,EUR2005,NaN,CH,Current,Business-as-Usual,...,straw,kg,"01913: Cereal straw, husks, unprepared, ground...",0.85,1,ce21e796-ac1a-4fcf-a8b4-8170c039516c,0,1996-01-01,2018-12-31,"barley production, Swiss integrated production..."
b45ebf7e-9c37-43af-8031-3d52ff5dfe8f_a235b2ff-3237-44b0-a445-b852376a1939,b45ebf7e-9c37-43af-8031-3d52ff5dfe8f,a235b2ff-3237-44b0-a445-b852376a1939,"barley production, Swiss integrated production...","0111:Growing of cereals (except rice), legumin...",0.117,EUR2005,NaN,CH,Current,Business-as-Usual,...,"barley grain, Swiss integrated production",kg,"01152: Barley, other",0.85,1,ce21e796-ac1a-4fcf-a8b4-8170c039516c,0,1996-01-01,2018-12-31,"barley production, Swiss integrated production..."


## 2.2. `A` "technology matrix"

In [19]:
ecoinvent['A'].shape

(16022, 16022)

In [20]:
ecoinvent['A'].head(5)

index,a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,8292bb2c-903f-400e-861e-fa427f39e262_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,60b530e4-1f8c-44bc-aa12-5cd4d2187dae_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,558a3696-1009-44a7-8753-da7217f46c77_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,9d4929b6-4d6b-41ed-ba7e-0abd6c974867_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,c8c5da0c-49af-43d7-82a8-5d46d66f495c_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,...,bd1d1d91-fc01-4b4c-ad58-858c00bab907_8d7c013a-5aca-44d7-8fc4-193844d2953a,5528611b-1e96-4e7e-b5ea-a103534074ba_0baac448-32a1-4b9f-afec-69a9b9b203ac,c60bcfeb-d21f-4d2e-915b-a5bb52a1929f_f70c1877-6db5-4024-b260-69f50314b40c,3c138178-0520-4950-af40-23fce8501056_fd27af60-7b68-4549-bf35-022eacd74f93,d448c7f9-cd47-4456-8a32-0d0e8e024c14_ad38bbd2-e41c-4509-9dc3-c5c708a36606,31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,73c9e29c-885e-490e-95f3-087227443758_101e55f2-2da0-528b-96fd-21687e242c1c,91ed532a-08f2-4fdd-b5e9-28bb2afd1336_451550a8-d9e6-4396-9f9a-97e752111228
index,,,,,,,,,,,,,,,,,,,,,
a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
A: pd.DataFrame = ecoinvent['A']

## 2.2. `F` ('`B`' in _Comp. Struct. of LCA_) "intervention matrix"

In [12]:
ecoinvent['F'].shape

(4243, 16022)

In [13]:
ecoinvent['F'].head(5)

index,a96cb241-a4a9-4980-a16a-ba4b6a80175e_aeaf5266-3f9c-4074-bd34-eba76a61760c,6885fd40-ff73-40a4-8f71-225577ec684e_aeaf5266-3f9c-4074-bd34-eba76a61760c,d1c3b8ad-58b6-4ea4-8877-a51b356556bc_281fc4f0-c05d-410a-a784-06e3508e78e6,92068396-88c7-45ed-9008-622008a299f3_0d860eb4-1a25-41b4-a821-81f5726d86e5,35ddb020-9812-4808-bdfb-6845a454a73c_0d860eb4-1a25-41b4-a821-81f5726d86e5,8292bb2c-903f-400e-861e-fa427f39e262_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,60b530e4-1f8c-44bc-aa12-5cd4d2187dae_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,558a3696-1009-44a7-8753-da7217f46c77_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,9d4929b6-4d6b-41ed-ba7e-0abd6c974867_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,c8c5da0c-49af-43d7-82a8-5d46d66f495c_3f6dada9-2497-4e1c-9e1b-eabafa6920f8,...,bd1d1d91-fc01-4b4c-ad58-858c00bab907_8d7c013a-5aca-44d7-8fc4-193844d2953a,5528611b-1e96-4e7e-b5ea-a103534074ba_0baac448-32a1-4b9f-afec-69a9b9b203ac,c60bcfeb-d21f-4d2e-915b-a5bb52a1929f_f70c1877-6db5-4024-b260-69f50314b40c,3c138178-0520-4950-af40-23fce8501056_fd27af60-7b68-4549-bf35-022eacd74f93,d448c7f9-cd47-4456-8a32-0d0e8e024c14_ad38bbd2-e41c-4509-9dc3-c5c708a36606,31e6ab74-141d-49f4-9e75-5974bd62742b_b3184435-d00c-5713-823d-5ff6741bfbbc,b50b032c-8bf0-41a4-bb7a-ce5e12cabf01_d13ae29b-7d6a-55d2-8380-86343a80fed8,307b681d-c6bc-44cb-ab3e-a9067a4e115a_ed25f589-c6c8-56a7-a518-b01dd3e8a453,73c9e29c-885e-490e-95f3-087227443758_101e55f2-2da0-528b-96fd-21687e242c1c,91ed532a-08f2-4fdd-b5e9-28bb2afd1336_451550a8-d9e6-4396-9f9a-97e752111228
id,,,,,,,,,,,,,,,,,,,,,
d21da01e-f96f-4db5-9746-7b70db8a1f2c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83bafcf1-2f2e-4a32-89a0-f1f16ca10626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
09db39be-d9a6-4fc3-8d25-1f80b23e9131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38a622c6-f086-4763-a952-7c6b3b1c42ba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.4. `C` "characterization factors"

In [14]:
ecoinvent['C'].shape

(879, 4243)

In [15]:
ecoinvent['C'].head(5)

id,d21da01e-f96f-4db5-9746-7b70db8a1f2c,90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,83bafcf1-2f2e-4a32-89a0-f1f16ca10626,09db39be-d9a6-4fc3-8d25-1f80b23e9131,38a622c6-f086-4763-a952-7c6b3b1c42ba,9dd01d5b-3677-4822-9cd4-36d21b0e23d1,b78e77cb-7636-4420-855e-17239984f8b3,cc9a442f-c96a-4bdc-990d-8b58f72b4e07,048baf1e-6cdc-44a5-92e2-32d15ff54885,541a823c-0aad-4dc4-9123-d4af4647d942,...,0f3a98cd-b692-4801-86b5-b2222a953417,a65308df-73ac-4aa8-88f0-319743a50293,c5b4869c-f131-4a8c-911f-f252cafb2e30,77fd3c56-5c3e-4070-93fa-de03fa455d47,3d59d864-b6c3-4315-ab21-2c2ba2119ad1,e9eb024b-6fba-4dce-b280-9415b2ba4695,d0aaa60f-40d9-4c12-a486-954333aa549f,cc415ec5-ef9b-4239-b423-32906818e457,a66849fd-060a-40b5-bd9d-04caa632b75c,365dffb2-9e7a-44fa-a512-8c851fc7094c
impact_label,,,,,,,,,,,,,,,,,,,,,
"EDIP; environmental impact; global warming, GWP 100a; kg CO2-Eq",2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"EDIP; environmental impact; global warming, GWP 20a; kg CO2-Eq",2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"EDIP; environmental impact; global warming, GWP 500a; kg CO2-Eq",2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"EDIP; environmental impact; photochemical ozone formation, high NOx POCP; kg ethyle.",0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"EDIP; environmental impact; photochemical ozone formation, low NOx POCP; kg ethyle.",0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.5. `IMP` "impact categories"

In [16]:
ecoinvent['IMP'].shape

(879, 4)

In [21]:
ecoinvent['IMP'].head(15)

,method,category,indicator,impact_score_unit
impact_label,,,,
"EDIP; environmental impact; global warming, GWP 100a; kg CO2-Eq",EDIP,environmental impact,"global warming, GWP 100a",kg CO2-Eq
"EDIP; environmental impact; global warming, GWP 20a; kg CO2-Eq",EDIP,environmental impact,"global warming, GWP 20a",kg CO2-Eq
"EDIP; environmental impact; global warming, GWP 500a; kg CO2-Eq",EDIP,environmental impact,"global warming, GWP 500a",kg CO2-Eq
"EDIP; environmental impact; photochemical ozone formation, high NOx POCP; kg ethyle.",EDIP,environmental impact,"photochemical ozone formation, high NOx POCP",kg ethyle.
"EDIP; environmental impact; photochemical ozone formation, low NOx POCP; kg ethyle.",EDIP,environmental impact,"photochemical ozone formation, low NOx POCP",kg ethyle.
EDIP2003; photochemical ozone formation; impacts on human health; person.pp.,EDIP2003,photochemical ozone formation,impacts on human health,person.pp.
EDIP2003; photochemical ozone formation; impacts on vegetation; m2.ppm.h,EDIP2003,photochemical ozone formation,impacts on vegetation,m2.ppm.h
USEtox; ecotoxicity; total; CTU,USEtox,ecotoxicity,total,CTU
"eco-indicator 99, (E,E); human health; respiratory effects; points","eco-indicator 99, (E,E)",human health,respiratory effects,points


## 2.6. `STR` "stressors metadata"

In [18]:
ecoinvent['STR'].shape

(4243, 6)

In [20]:
ecoinvent['STR'].head(15)

,id,name,unit,cas,comp,subcomp
id,,,,,,
d21da01e-f96f-4db5-9746-7b70db8a1f2c,d21da01e-f96f-4db5-9746-7b70db8a1f2c,"1,4-Butanediol",kg,000110-63-4,air,"low population density, long-term"
90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,90653a29-2f53-4b1b-88bd-9ae2fe64a8d6,"1,4-Butanediol",kg,000110-63-4,air,lower stratosphere + upper troposphere
83bafcf1-2f2e-4a32-89a0-f1f16ca10626,83bafcf1-2f2e-4a32-89a0-f1f16ca10626,"1,4-Butanediol",kg,000110-63-4,air,non-urban air or from high stacks
09db39be-d9a6-4fc3-8d25-1f80b23e9131,09db39be-d9a6-4fc3-8d25-1f80b23e9131,"1,4-Butanediol",kg,000110-63-4,air,unspecified
38a622c6-f086-4763-a952-7c6b3b1c42ba,38a622c6-f086-4763-a952-7c6b3b1c42ba,"1,4-Butanediol",kg,000110-63-4,air,urban air close to ground
9dd01d5b-3677-4822-9cd4-36d21b0e23d1,9dd01d5b-3677-4822-9cd4-36d21b0e23d1,1-Pentanol,kg,000071-41-0,air,"low population density, long-term"
b78e77cb-7636-4420-855e-17239984f8b3,b78e77cb-7636-4420-855e-17239984f8b3,1-Pentanol,kg,000071-41-0,air,lower stratosphere + upper troposphere
cc9a442f-c96a-4bdc-990d-8b58f72b4e07,cc9a442f-c96a-4bdc-990d-8b58f72b4e07,1-Pentanol,kg,000071-41-0,air,non-urban air or from high stacks
048baf1e-6cdc-44a5-92e2-32d15ff54885,048baf1e-6cdc-44a5-92e2-32d15ff54885,1-Pentanol,kg,000071-41-0,air,unspecified
